# Inference with your model

In a [previous tutorial](train_your_first_model.ipynb), you successfully trained your model.
Now, you learn how to use your model to classify a handwritten digit image. You will learn how to implement a `Translator` interface to convert between POJO and `NDArray`.


## Preparation

This tutorial requires the installation of Java Kernel. To install the Java Kernel, see the [README](https://github.com/awslabs/djl/blob/master/jupyter/README.md).

In [ ]:
%maven ai.djl:api:0.2.0
%maven ai.djl:repository:0.2.0
%maven ai.djl:model-zoo:0.2.0
%maven ai.djl.mxnet:mxnet-engine:0.2.0
%maven ai.djl.mxnet:mxnet-model-zoo:0.2.0
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26
%maven net.java.dev.jna:jna:5.3.0

### Include MXNet engine dependency

This tutorial uses MXNet engine as its backend. MXNet has different [build flavor](https://mxnet.apache.org/get_started?version=v1.5.1&platform=linux&language=python&environ=pip&processor=cpu) and it is platform specific.
Please read [here](https://github.com/awslabs/djl/blob/master/examples/README.md#engine-selection) for how to select MXNet engine flavor.

In [ ]:
String classifier = System.getProperty("os.name").startsWith("Mac") ? "osx-x86_64" : "linux-x86_64";

%maven ai.djl.mxnet:mxnet-native-mkl:jar:${classifier}:1.6.0-a

In [ ]:
import java.awt.image.*;
import java.nio.file.*;
import java.util.*;
import java.util.stream.*;
import ai.djl.*;
import ai.djl.inference.*;
import ai.djl.ndarray.*;
import ai.djl.ndarray.index.*;
import ai.djl.modality.*;
import ai.djl.modality.cv.*;
import ai.djl.modality.cv.util.*;
import ai.djl.modality.cv.util.NDImageUtils.Flag;
import ai.djl.mxnet.zoo.*;
import ai.djl.translate.*;
import ai.djl.util.*;
import ai.djl.zoo.cv.classification.*;

## Step 1: Load your handwritten digit image

In [ ]:
var img = BufferedImageUtils.fromUrl("https://djl-ai.s3.amazonaws.com/resources/images/0.png");
img

## Step 2: Load your model

In [ ]:
Path modelDir = Paths.get("build/mlp");
Model model = Model.newInstance();
model.setBlock(new Mlp(28, 28));
model.load(modelDir);

## Step 3: Create a `Translator`

In [ ]:
Translator<BufferedImage, Classifications> translator = new Translator<BufferedImage, Classifications>() {

    @Override
    public NDList processInput(TranslatorContext ctx, BufferedImage input) {
        // Convert BufferedImage to NDArray
        NDArray array = BufferedImageUtils.toNDArray(ctx.getNDManager(), input, NDImageUtils.Flag.GRAYSCALE);
        return new NDList(NDImageUtils.toTensor(array));
    }

    @Override
    public Classifications processOutput(TranslatorContext ctx, NDList list) {
        NDArray probabilities = list.singletonOrThrow().softmax(0);
        List<String> indices = IntStream.range(0, 10).mapToObj(String::valueOf).collect(Collectors.toList());
        return new Classifications(indices, probabilities);
    }
};



## Step 3: Run inference

In [ ]:
var predictor = model.newPredictor(translator);
var classifications = predictor.predict(img);

classifications

## Summary


You can find the full source in the [examples project](https://github.com/awslabs/djl/blob/master/examples/docs/image_classification.md).